In [1]:
import numpy as np

from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from scipy import stats

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F
torch.manual_seed(1)

import time
import datetime

from collections import defaultdict
from collections import Counter

from random import randint
import pickle
import nltk
import string
puncs = set(string.punctuation)

In [2]:
min_count = 2
threshold_for_subsampling = 0.00001 #recommended in Mikolov 2013
neg_context_k = 5 #negative samples
embedding_dim = 100
batch_size = 50
epochs = 10
lr = 0.01
window_size = 5 #dynamic given this value

#removed punctuation marks, removed infrequent ones, subsampled frequent ones
#negative samples done 

#what should be the output? which candidates should be selected?
#what to do for word embedding initialization
# Improving Distributional Similarity with Lessons Learned fromWord Embeddings
# Omer Levy Yoav Goldberg Ido Dagan Computer

In [3]:
dataset = 'wa/dev.en'

#read the files
with open(dataset) as f:
    sentences = [l.strip() for l in f.readlines()]

#get all the tokens from the corpus

tokens_list = []
sentence_list = []
for s in sentences:
    split_sent = s.split()
    sentence = []
    for w in split_sent:
        
        tokens_list.append(w)
        sentence.append(w)
#         #filter stopwords
#         if w not in stopwords:
#             tokens_list.append(w)
#             sentence.append(w)
    
    sentence_list.append(sentence)
    
tokens = list(sorted(set(tokens_list)))
print('Vocabulary size', len(tokens))

count_tokens = Counter(tokens_list)

#words appearing fewer than this are not considered words or contexts
#subsample frequent words

temp_sentence_list = []

for s in range(len(sentence_list)):
    temp_sentence_list.append(sentence_list[s])

#Find the infrequent words
for s in range(len(temp_sentence_list)):
    for w in range(len(temp_sentence_list[s])):
        word = temp_sentence_list[s][w]
        if count_tokens[word] < min_count:
            sentence_list[s][w] = '<unk>'
            
            #remove the infrequent words from tokens
            tokens.remove(word)

#we prune rare words altogether

#tokens.append('<unk>')
vocab_size = len(tokens)
print('After rare word pruning', vocab_size)
#remove the infrequent words from the sentences
for s in sentence_list:
    s[:] = [w for w in s if w != '<unk>' and w not in puncs]

flat_token_list = []

for s in sentence_list:
    for w in s:
        flat_token_list.append(w)

Vocabulary size 322
After rare word pruning 84


In [4]:
subsample_probs = defaultdict(float) 

count_words = Counter(flat_token_list)
print(count_words)
print(sum(count_words.values()))

for t in count_words:
    
    #Distributed Representations of Words and Phrases and their Compositionality
    #Mikolov 2013
    #p = 1 - sqrt(t/f)
    subsample_probs[t] = 1 - np.sqrt(threshold_for_subsampling/(count_words[t]/len(flat_token_list)))
    
temp_sentence_list = []

for s in range(len(sentence_list)):
    temp_sentence_list.append(sentence_list[s])

subsampled_sentence_list = []

#Prune frequent words
for s in range(len(temp_sentence_list)):
    sent = []
    for w in range(len(temp_sentence_list[s])):
        
        word = temp_sentence_list[s][w]
        word_prob = subsample_probs[word]
        
        rand_prob = np.random.rand(1)[0]
       
        if rand_prob < word_prob:
            #keep word
            sent.append(temp_sentence_list[s][w])
    
    subsampled_sentence_list.append(sent)

Counter({'the': 33, 'to': 23, 'of': 14, 'in': 14, 'and': 13, 'that': 12, 'for': 11, 'it': 11, 'we': 11, 'a': 10, 'have': 9, 'is': 7, 'they': 7, 'be': 7, 'are': 6, 'I': 6, 'as': 6, 'very': 5, 'not': 5, 'our': 5, 'been': 5, 'this': 5, 'do': 4, 'what': 4, 'them': 3, 'which': 3, 'people': 3, 'on': 3, 'by': 3, 'one': 3, 'does': 3, 'Bill': 3, 'has': 3, 'there': 3, 'public': 3, 'or': 3, 'at': 3, 'however': 3, 'would': 3, 'want': 3, 'no': 3, 'will': 3, 'Mr.': 3, 'women': 3, 'complex': 2, 'whole': 2, 'most': 2, 'today': 2, 'who': 2, 'effective': 2, 'so': 2, 'let': 2, 'think': 2, 'should': 2, 'look': 2, 'cost': 2, 'some': 2, 'given': 2, 'see': 2, 'up': 2, 'say': 2, 'than': 2, 'programs': 2, 'within': 2, 'country': 2, 'support': 2, 'make': 2, 'Prime': 2, 'Minister': 2, 'need': 2, 'action': 2, 'under': 2, 'Speaker': 2, 'pretends': 2, 'employment': 2, 'equity': 2, 'job': 2, 'growers': 2, 'begin': 2})
363


In [5]:
pickle.dump(subsampled_sentence_list, open('subsampled_dataset.p', 'wb'))

In [6]:
subsampled_sentence_list = pickle.load(open('subsampled_dataset.p', 'rb'))

In [7]:
flat_subsampled_token_list = []

for s in subsampled_sentence_list:
    for w in s:
        flat_subsampled_token_list.append(w)

unigram_counts = Counter(flat_subsampled_token_list)

unigram_probs = defaultdict(float) 

for t in unigram_counts:
    unigram_probs[t] = unigram_counts[t]/len(flat_subsampled_token_list)

pickle.dump(unigram_probs, open('unigram_probs.p', 'wb'))

In [8]:
unigram_probs = pickle.load(open('unigram_probs.p', 'rb'))

In [9]:
def sampled_window(sentence, central_word_index, window_size):
    
    #implicit weighing of the context words
    #dynamic context window size for each word
    dynamic_window_size = randint(1,window_size)
    
    #dws = 2, i-2,i-1,i+1,i+2
    
    pairs = []
    word = sentence[central_word_index]
    for w in range(1,dynamic_window_size+1):
        
        left_cont = central_word_index - w
        right_cont = central_word_index + w
        
        #find the window words to the left and right
        #add as pair if they are inside sentence boundaries
        
        if left_cont > -1:
            pairs.append((word, sentence[left_cont]))
        
        if right_cont < len(sentence):
            pairs.append((word, sentence[right_cont]))
        
    return pairs

In [10]:
neg_sample_probs = defaultdict(float)

neg_normalizer = 0.0

for t in unigram_probs:
    neg_sample_probs[t] = np.power(unigram_probs[t], 0.75)
    neg_normalizer += neg_sample_probs[t]
    
for n in neg_sample_probs:
    neg_sample_probs[n] = neg_sample_probs[n] / neg_normalizer

In [11]:
#à la Mikolov
#table to pick negative context words
#fill the table with each word, count = negsampling prob*table size
neg_table_size = 10000000

unigram_table = np.empty(neg_table_size, dtype=object)
i = 0

for n in neg_sample_probs:
    count = int(neg_sample_probs[n] * neg_table_size)
    
    for c in range(count):
        unigram_table[i] = n
        
        i += 1
    
unigram_table = unigram_table[unigram_table != np.array(None)]
np.random.shuffle(unigram_table)
print(i)
print(len(unigram_table))
print(unigram_table[0:10])

9999965
9999965
['given' 'has' 'is' 'at' 'most' 'is' 'are' 'the' 'in' 'in']


In [12]:
def pick_neg_context(table, k):
    
    neg_samples = []
    
    for i in range(k):
        
        r = randint(0,len(table)-1)
        neg_samples.append(table[r])
        
    return neg_samples    

In [13]:
pick_neg_context(unigram_table, 5)

['I', 'has', 'no', 'Bill', 'will']

In [14]:
pos_context_pairs = []
neg_context_samples = []

for sentence in sentence_list:
    for w in range(len(sentence)):
        
        pos_context_pairs.extend(sampled_window(sentence, w, window_size))
              
for p in range(len(pos_context_pairs)):
    
    pair = pos_context_pairs[p]
    neg_context_samples.append((pair[0], pick_neg_context(unigram_table, neg_context_k)))

pickle.dump(pos_context_pairs, open('pos_context_pairs.p', 'wb'))
pickle.dump(neg_context_samples, open('neg_context_samples.p', 'wb'))

In [15]:
print(len(pos_context_pairs), len(neg_context_samples))
print(pos_context_pairs[0], neg_context_samples[0])

1682 1682
('of', 'them') ('of', ['begin', 'I', 'has', 'effective', 'the'])


In [16]:
pos_context_pairs = pickle.load(open('pos_context_pairs.p', 'rb'))
neg_context_samples = pickle.load(open('neg_context_samples.p', 'rb'))

In [17]:
tokens = list(set(flat_subsampled_token_list))

#default dictionary key:id value:token
id2tokens = defaultdict(str)
                        
for i in range(len(tokens)):
    id2tokens[i] = tokens[i]
    
#default dictionary key:token value:id
tokens2id = defaultdict(int)

for ind in id2tokens:
    tokens2id[id2tokens[ind]] = ind
    
vocabulary_size = len(tokens2id)
print(vocabulary_size)

79


In [18]:
#convert dataset to ids
pos_data = []
neg_data = []

for p in pos_context_pairs:
    pos_data.append((tokens2id[p[0]], tokens2id[p[1]]))
    
for n in neg_context_samples:
    
    word_id = tokens2id[n[0]]
    neg_samples = n[1]
    
    neg_ids = []
    
    for ns in neg_samples:
        neg_ids.append(tokens2id[ns])
        
    neg_data.append((word_id, neg_ids))

In [19]:
print(len(pos_data), len(neg_data), len(pos_context_pairs), len(neg_context_samples))

1682 1682 1682 1682


In [20]:
print(pos_data[0], pos_context_pairs[0], neg_context_samples[0], neg_data[0])

(66, 42) ('of', 'them') ('of', ['begin', 'I', 'has', 'effective', 'the']) (66, [3, 25, 22, 12, 73])


In [21]:
pickle.dump(pos_data, open('pos_data.p', 'wb'))
pickle.dump(neg_data, open('neg_data.p', 'wb'))

In [22]:
with open('pos_data.p', 'rb') as f:
    pos_data = pickle.load(f)
    
with open('neg_data.p', 'rb') as f:
    neg_data = pickle.load(f)

In [23]:
neg_data

[(66, [3, 25, 22, 12, 73]),
 (42, [66, 31, 20, 62, 45]),
 (42, [68, 10, 28, 10, 15]),
 (42, [73, 73, 38, 61, 58]),
 (42, [31, 68, 37, 37, 26]),
 (4, [15, 16, 28, 26, 73]),
 (4, [27, 25, 27, 32, 22]),
 (4, [56, 4, 25, 73, 34]),
 (4, [75, 7, 25, 7, 62]),
 (21, [37, 21, 3, 66, 4]),
 (21, [44, 5, 6, 77, 41]),
 (21, [60, 73, 67, 77, 25]),
 (21, [37, 46, 73, 18, 27]),
 (21, [34, 50, 39, 22, 74]),
 (21, [11, 35, 28, 0, 30]),
 (21, [44, 49, 62, 51, 44]),
 (8, [19, 60, 53, 68, 73]),
 (8, [27, 66, 73, 38, 69]),
 (8, [23, 63, 27, 21, 18]),
 (8, [32, 38, 63, 73, 73]),
 (8, [7, 37, 37, 63, 62]),
 (8, [60, 68, 38, 25, 68]),
 (8, [73, 14, 23, 23, 44]),
 (8, [50, 37, 46, 46, 66]),
 (73, [26, 11, 41, 37, 18]),
 (73, [26, 40, 38, 19, 71]),
 (73, [64, 27, 29, 10, 20]),
 (73, [20, 26, 5, 66, 35]),
 (73, [1, 73, 69, 5, 73]),
 (73, [12, 26, 2, 63, 19]),
 (73, [65, 62, 73, 26, 23]),
 (73, [73, 66, 46, 30, 33]),
 (73, [66, 27, 71, 60, 37]),
 (73, [77, 37, 28, 51, 30]),
 (73, [0, 63, 74, 64, 3]),
 (73, [24, 70